# 

This is based on https://learn.deeplearning.ai/langchain/lesson/4/chains lecture

In [ ]:
from langchain.document_loaders.github import GitHubIssuesLoader
import os

from dotenv import load_dotenv

load_dotenv()

GHA_TOKEN = os.getenv('GHA')
os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory.buffer import ConversationBufferMemory
chat_llm = ChatOpenAI(temperature=0.8)
memory =ConversationBufferMemory()

LLM chain is one of the simplest and just connects a LLM and Prompt

In [ ]:
data = [

{"product":"Apple Mac", "review": "really excellent and fast"},
{"product":"Windward isles bananas", "review": "fresh and tasty"},
{"product":"Cat scarer", "review": "Didn't work, the cats paid it no attention"},
{"product":"Dog collar", "review": "Our dog liked it but slipped his collar and escaped"},
]

In [ ]:
getCompanyNameT = PromptTemplate.from_template("For a product {product} think of a good company name in a {style} style")
chain = LLMChain(llm=chat_llm, prompt=getCompanyNameT,verbose=True)

In [ ]:
chain.run(product=data[1]['product'], style="jokey")

The next is a simple sequential chain where we can put the output of 1 message into the input of the next

In [ ]:
from langchain.chains import SimpleSequentialChain
## only works with single input and output
getCompanyNameT = PromptTemplate.from_template("For a product {product} think of a good company name in comedy style")
getCompanyTaglineT = PromptTemplate.from_template("For a companyName {companyName} think of a good tagline")
## we don't need to worry about input/output names as there is only 1
chain1 = LLMChain(llm=chat_llm, prompt=getCompanyNameT,verbose=True)
chain2 = LLMChain(llm=chat_llm, prompt=getCompanyTaglineT,verbose=True)

sequential_chain = SimpleSequentialChain(chains=[chain1, chain2])
resp=sequential_chain.run(data[0]['product'])


Question - how to get intermediate outputs?

In [ ]:
resp

Now let's use a regular Sequential Chain to use multiple inputs and outputs.
We need to define the output keys of each subchain

For the sequential chain wrapper we need to define the first 2 inputs and final output.

In [ ]:
from langchain.chains import SequentialChain
getCompanyNameT = PromptTemplate.from_template("For a product {product} think of a good company name in {style} style")
getCompanyTaglineT = PromptTemplate.from_template("For a companyName {companyName} think of a good tagline")
sentimentT = PromptTemplate.from_template("for a tagline {tagline} return the sentiment which must be one of Positive, Neutral or Negative")
summaryT = PromptTemplate.from_template("Summarise whether {companyName} with tagline {tagline} and sentiment {sentiment} is a good name")

llm_chain1 = LLMChain(llm=chat_llm, prompt=getCompanyNameT,verbose=True,output_key="companyName")
llm_chain2 = LLMChain(llm=chat_llm, prompt=getCompanyTaglineT,verbose=True, output_key="tagline")
llm_chain3 = LLMChain(llm=chat_llm, prompt=sentimentT,verbose=True, output_key="sentiment")
llm_chain4 = LLMChain(llm=chat_llm, prompt=summaryT,verbose=True, output_key="summary")
sub_chains = [llm_chain1,llm_chain2,llm_chain3,llm_chain4]
seq_chain = SequentialChain(chains=sub_chains, input_variables=["product", "style"],output_variables=["summary"] )
seq_chain.run(product=data[3]['product'], style="silly")


Now we'll use router chain

In [ ]:
## create some prompts 
physics_promptT = """
 You are highly knowledgable about physics and are able to give detailed answers  about physics.
 Here is a question: {input}
 """
english_promptT = """
 You are highly knowledgable about english literature and have read many classic texts.
  You  are able to give detailed answers  about english novels.
 Here is a question: {input}
 """
spanish_promptT = """
 You are highly knowledgable about spanish grammar and language and are able to give detailed explanations
 based on your years of experience conversing in Spanish.
 Here is a question: {input}
 """
generic_promptT = """
 Here is a question: {input}
 """

In [ ]:
## this is used to create the router prompt
prompt_info = [
 {
    "name":"physics_prompt",
     "description" : "good  for answering questions about physics",
     "prompt_template":physics_promptT
 },
{
    "name":"english_prompt",
     "description" : "good  for answering questions about english literature",
     "prompt_template":english_promptT
 },  
 {
    "name":"spanish_prompt",
     "description" : "good  for answering questions about spanish",
     "prompt_template":spanish_promptT
 }
]

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router import multi_prompt_prompt
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

destination_chains = {}
for prompt in prompt_info:
    name = prompt['name']
    templateS = prompt['prompt_template']
    promptT = ChatPromptTemplate.from_template(templateS)
    llm_chain = LLMChain(llm=chat_llm, prompt=promptT)
    destination_chains[name]=llm_chain
default_chain = LLMChain(llm=chat_llm, prompt=ChatPromptTemplate.from_template("{{input}}"))

## This goes into the router template as a choice of routers to use
destinations = [f"{p['name']}:   {p['description']}" for p in prompt_info]
destination_string= '\n'.join(destinations)

In [ ]:
## partially fill in the main template
router_templateS = multi_prompt_prompt.MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_string)

## now create the router prompt and add it to router chain
router_prompt = PromptTemplate(template=router_templateS, input_variables=["input"], output_parser=RouterOutputParser())

In [ ]:
router_chain = LLMRouterChain.from_llm(chat_llm, router_prompt)

In [ ]:

## this is the final chain.
top_chain = MultiPromptChain(destination_chains=destination_chains, verbose=True,router_chain=router_chain,default_chain=default_chain)
top_chain.run("tell me about jane austen's book northanger abbey")